In [3]:
import os
import sys
import time
import json
import random
import groq
from huggingface_hub import HfFolder, Repository, create_repo, whoami
from huggingface_hub.utils import logging as hf_logging
from dotenv import load_dotenv
import os
import re
import math
from tqdm import tqdm
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, set_seed, BitsAndBytesConfig
from datasets import load_dataset, Dataset, DatasetDict
import matplotlib.pyplot as plt
import pickle
from testing import Tester
import os

In [4]:
# Let's avoid curating all our data again! Load in the pickle files:

with open('train.pkl', 'rb') as file:
    train = pickle.load(file)

with open('test.pkl', 'rb') as file:
    test = pickle.load(file)

In [5]:
# A utility function to extract the price from a string

def get_price(s):
    s = s.replace('$','').replace(',','')
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    return float(match.group()) if match else 0

In [6]:
def messages_for(item):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "Price is $"}
    ]

In [7]:
client = groq.Client(
    api_key=os.environ.get("GROQ_API_KEY"),
)


GroqError: The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable

In [ ]:
def groq_llama_mini(item):
    try:
        response = client.chat.completions.create(
            model="llama3-70b-8192",
            messages=messages_for(item),
            max_tokens=5,
            temperature=0.0,
            seed=42
        )
        reply = response.choices[0].message.content.strip()
        return get_price(reply)
    except Exception as e:
        print(f"Groq API Error: {e}")
        return None

In [ ]:
Tester.test(groq_llama_mini, test)

NameError: name 'Tester' is not defined